## some documentation
- process slang:
    * https://github.com/vi3k6i5/flashtext1
    * https://github.com/cjhutto/vaderSentiment
    * https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f

In [1]:
!pip install nltk 
!pip install stanza
!pip install spacy
!spacy download en_core_web_sm # sm md lg
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
# nltk
import nltk
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

STOPWORDS = set(stopwords.words('english'))
NLTK_WORDS = set(words.words())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
def nltk_lemma(text):
    lemmatizer = WordNetLemmatizer()
    lemmatizer.lemmatize(text)

In [4]:
# Stanza NLP
import stanza

stanza.download('en', package='ewt', processors='tokenize,mwt,pos,lemma', verbose=True)
stNLP = stanza.Pipeline(processors='tokenize,mwt,pos,lemma',
                      lang='en',
                      use_gpu=True)

2020-08-26 22:43:58 WARNING: Can not find mwt: ewt from official model list. Ignoring it.
2020-08-26 22:43:58 INFO: Downloading these customized packages for language: en (English)...
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| pretrain  | ewt     |

2020-08-26 22:43:58 INFO: File exists: /root/stanza_resources/en/tokenize/ewt.pt.
2020-08-26 22:43:58 INFO: File exists: /root/stanza_resources/en/pos/ewt.pt.
2020-08-26 22:43:58 INFO: File exists: /root/stanza_resources/en/lemma/ewt.pt.
2020-08-26 22:43:59 INFO: File exists: /root/stanza_resources/en/pretrain/ewt.pt.
2020-08-26 22:43:59 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-08-26 22:43:59 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-08-26 22:43:59 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt  

In [5]:
# stanza
def stanza_lemma(text):
    doc = stNLP(text)
    return ' '.join([word.lemma for sent in doc.sentences for word in sent.words])

In [6]:
# Spacy NLP
import spacy
spNLP = spacy.load('en_core_web_sm')
spNLP.max_length = 103950039 # or higher
# spacy.prefer_gpu() #will not work with stanza

### **VADER**
* VADER, has different ratings depending on the form of the word and therefore the input should not be stemmed or lemmatized.

* disadvantage of this approach is that Out of Vocab (OOV) words that the sentiment analysis tool has not seen before will not be classified as positive/negative (e.g. typos).

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
SIA = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### **TextBlob**
* use a bag of words classifier, but the advantage is that it includes subjetivity analysis (factual/opinated)
* it doesn't contain the heuristics that nltk has, it won't intensify or negate a sentence's sentiment

* will return the subjectivity of the text

In [8]:
from textblob import TextBlob

### **Flair**
* classifier based on a character-leval LSTM. Takes a sequences of letters and words into account when predicting

* one of its biggest advantages is that it can predict a sentiment for OOV words that it has never seen before too (such as typos)

In [25]:
!pip3 install flair
# tiene en cuenta secuencias de letras y palabras al predecir
import flair
flair_sent = flair.models.TextClassifier.load('en-sentiment')

2020-08-26 23:03:28,035 loading file /root/.flair/models/sentiment-en-mix-distillbert.pt


In [10]:
def flair_lstm(text):
    x = flair.data.Sentence(text)
    flair_sent.predict(sentences=x)
    total_sent = x.labels
    return total_sent 

In [11]:
# labels
highlights = {
    # related with speech recognition
    'professional qualities': ['handles pressure'],
    'soft skills': ['silence'],
    'answer analysis': ['filler words', 'long pause', 'focus', 'patience'], 

    'polarities': {
        'negative': [
                     # confidence
                     'not confident', 
                     'unsure',

                     # professional qualities
                     '',
                     'disordered',
                     'talkative',
                     'uninsterested', # 'engaged'

                     # soft skills
                     'sad',
                     'unfriendly'
                     ],

        'positive': [
                     # confidence
                     'confident', 
                     'certany',

                     # professional qualities
                     'handles pressure',
                     'organized',
                     'concise', 
                     'interested', # 'engaged'

                     # soft skills
                     'happy',
                     'friendly'
                     ]
    }
}

main_lst = list(highlights.values())
main_labels = [k for j in main_lst for k in j]

neg_pos_lst = highlights['polarities'].values()
neg_pos_labels = [k for j in neg_pos_lst for k in j]

In [12]:
neg_pos_lst

dict_values([['not confident', 'unsure', '', 'disordered', 'talkative', 'uninsterested', 'sad', 'unfriendly'], ['confident', 'certany', 'handles pressure', 'organized', 'concise', 'interested', 'happy', 'friendly']])

In [13]:
neg_pos_labels

['not confident',
 'unsure',
 '',
 'disordered',
 'talkative',
 'uninsterested',
 'sad',
 'unfriendly',
 'confident',
 'certany',
 'handles pressure',
 'organized',
 'concise',
 'interested',
 'happy',
 'friendly']

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import re

In [15]:
def load_data(data_path=None):
    print('load the dataset...\n')
    !mkdir -p data
    !wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip -P data
    !unzip -n -d data data/training.1600000.processed.noemoticon.csv.zip

In [32]:
def preprocess_dataset(PATH_FILE, index_col=None):
    print('preprocess the dataset...\n')

    # load_data
    load_data()
    print('Database loaded\n')

    # cleaning data
    unclean_df = pd.read_csv(PATH_FILE,
                     names=['polarity', 'id', 'date', 'query', 'user', 'text'],
                     encoding='latin-1') # if utf-8: UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 232719-232720: invalid continuation byte

    # replace polarity
    unclean_df.polarity = unclean_df.polarity.replace({0: 0, 4: 1}) 
    
    # dropping unneeded columns
    unclean_df = unclean_df.drop(columns=['id', 'date', 'query', 'user']) 

    # lower case
    unclean_df['text'] = unclean_df['text'].str.lower()

    # removing urls
    unclean_df['text'] = unclean_df['text'].apply(lambda x: re.sub(r'https://www\.|http:\.|https://|www\.', '', x))
    unclean_df['text'] = unclean_df['text'].apply(lambda x: re.sub(r'[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil|cl)[\S]*\s?', '', x))

    # remove special character and numbers
    #unclean_df['text'] = unclean_df['text'].apply(lambda x: re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-zÁ-Úá-ú \t])|(\w+:\/\/\S+)|^rt|http.+?%', '', x))
    #unclean_df['text'] = unclean_df['text'].apply(lambda x: re.sub(r'\d+', '', x))

    # remove repetitions (goood ==> god)
    pattern = re.compile(r'(.)\1{2,}', re.DOTALL)
    unclean_df['text'] = unclean_df['text'].str.replace(pattern, r'\1')

    # removing empty values replacing them with nan
    nan_value = float('NaN')
    unclean_df.replace('', nan_value, inplace=True)
    unclean_df.dropna(inplace=True) # add subset

    # removing stopwords
    #df['text'] = df['text'].apply(lambda x: ' '.join([i for i in x.split() if i not in (STOPWORDS)]))

    # filtering and removing non-english words or misspelling
    #df['text'] = df['text'].apply(lambda x: ' '.join([i for i in x.split() if i.lower() in NLTK_WORDS or not i.isalpha()]))

    # rewritting the created file without NaN values
    unclean_df.to_csv('data/sentiment140-subset.csv', 
              quotechar='"', # check later!
              encoding='utf-8',
              index=False)

    # clean csv
    df = pd.read_csv('data/sentiment140-subset.csv', encoding='utf-8', warn_bad_lines=True).dropna()

    # checking if there's any NaN values
    isnull = [i for i in (df['text'].isnull()) if i == True]
    if isnull != []:
        sys.exit(0) # add response object here

    return df

In [33]:
df = preprocess_dataset(PATH_FILE='data/training.1600000.processed.noemoticon.csv')

preprocess the dataset...

load the dataset...

File ‘data/training.1600000.processed.noemoticon.csv.zip’ already there; not retrieving.

Archive:  data/training.1600000.processed.noemoticon.csv.zip
Database loaded



In [35]:
df.text[0]

"@switchfoot - aw, that's a bummer.  you shoulda got david carr of third day to do it. ;d"

In [19]:
def siaVader_compound(data):
    scores = SIA.polarity_scores(data)
    
    comp_score = scores['compound']
    if comp_score >= 0.05:
        str_comp = 'pos'
    elif comp_score <= -0.05:
        str_comp = 'neg'
    else: # (compound score > -0.05) and (compound score < 0.05)
        str_comp = 'neu'
    return str_comp

def siaVader_maxScore(data):
    scores = SIA.polarity_scores(data)
    
    del scores['compound']
    index = np.argmax(list(scores.values()))
    vader_MaxScore = list(scores.values())[index]
    vader_label = list(scores)[index]
    
    return vader_label

def siaVader_byWord(text):
    c = 0
    for n, y in enumerate(text):
        x = SIA.polarity_scores(y)
        if x['compound'] != 0.0:
            c += 1
            # print('{}. {} {}'.format(c, x, y))
            return 'pos' if x > 0.05 else 'neg'

In [20]:
df['comp_label'] = df['text'].apply(lambda x: siaVader_compound(data=x))

In [21]:
df['maxScore_label'] = df['text'].apply(lambda x: siaVader_maxScore(data=x))

In [22]:
df['comp_score'] = df['text'].apply(lambda x: SIA.polarity_scores(x)['compound'])

In [23]:
df['SIA'] = df['text'].apply(lambda x: SIA.polarity_scores(x))

In [24]:
df.tail(30)

,polarity,text,comp_label,maxScore_label,comp_score,SIA
1599921,1,thanks amp thanks that was just what i was l...,pos,neu,0.7003,"{'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compou..."
1599922,1,thanks martin not the most imaginative interf...,pos,neu,0.2382,"{'neg': 0.0, 'neu': 0.849, 'pos': 0.151, 'comp..."
1599923,1,congrats mike way to go,pos,neu,0.5267,"{'neg': 0.0, 'neu': 0.541, 'pos': 0.459, 'comp..."
1599924,1,omg office space i wanna steal it,neg,neu,-0.4939,"{'neg': 0.39, 'neu': 0.61, 'pos': 0.0, 'compou..."
1599925,1,ahaha nooo you were just away from everyone e...,neg,neu,-0.7717,"{'neg': 0.242, 'neu': 0.758, 'pos': 0.0, 'comp..."
1599926,1,hey im baack and thanks so much for all thos...,pos,neu,0.8316,"{'neg': 0.0, 'neu': 0.705, 'pos': 0.295, 'comp..."
1599927,1,yeah my conscience would be clear in that case,pos,neu,0.5859,"{'neg': 0.0, 'neu': 0.593, 'pos': 0.407, 'comp..."
1599928,1,thats my girl dishing out the quotadvicequot,neu,neu,0.0000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1599929,1,i second that,neu,neu,0.0000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1599930,1,in the garden,neu,neu,0.0000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
